# Customer-specialist
Agent responsible of solve fix to customer using requirementes describe on KB procedures

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/Foiac/customerSpecialist/refs/heads/main/customer-specialist/FlowDrawn.png" alt="Clean Data" style="width: 800px">
</div>

### Paths

In [57]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI
import os
from dotenv import load_dotenv

### Confs

In [58]:
load_dotenv()

# AI SEARCH ENVIRONMENT VARIABLES
SERVICE_NAME = os.getenv("AZURE_SEARCH_SERVICE_NAME")
INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")
API_KEY_AZURE_SEARCH = os.getenv("AZURE_SEARCH_API_KEY")

endpoint = f"https://{SERVICE_NAME}.search.windows.net"

# OPEN AI ENVIRONMENT VARIABLES
AZURE_ENDPOINT = os.getenv("AZURE_OPEN_AI_ENDPOINT")
API_VERSION = os.getenv("AZURE_OPEN_AI_API_VERSION")
API_KEY_AZURE_OPEN_AI = os.getenv("AZURE_OPEN_AI_API_KEY")


### Index Client

In [59]:
search_client = SearchClient(endpoint=endpoint, index_name=INDEX_NAME, credential=AzureKeyCredential(API_KEY_AZURE_SEARCH))

### OpenAI Client

In [60]:
client = AzureOpenAI(
    api_version=API_VERSION,
    azure_endpoint=AZURE_ENDPOINT,
    api_key=API_KEY_AZURE_OPEN_AI,
)

### Context function

In [61]:
def obter_contexto_rag(pergunta):
    resultados = search_client.search(pergunta, top=3)
    print("\n".join([doc['content'] for doc in resultados]))
    return "\n".join([doc['content'] for doc in resultados])

### Tools

In [62]:
tools = [{
    "type": "function",
    "function": {
        "name": "resetar_senha",
        "description": "Redefine a senha de um colaborador autorizado",
        "parameters": {
            "type": "object",
            "properties": {
                "cpf": {"type": "string"},
                "tipo_usuario": {"type": "string", "enum": ["colaborador", "gerente", "rh"]}
            },
            "required": ["cpf", "tipo_usuario"]
        }
    }
}]

### Prompt

In [63]:
prompt = """Você é um assistente virtual que ajuda os usuários conforme o procedimento descrito
            
            Todo procedimento te restrições que devem ser consideradas

            Ao executar o procedimento, você deve seguir o passo a passo do procedimento"""

### Define agent function

In [64]:
def chamar_agente(pergunta, contexto):
    resposta = client.chat.completions.create(
        model="gpt-4o-mini", 
        temperature=0,
        tools=tools,
        tool_choice="auto",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": pergunta},
            {"role": "system", "content": f"Base de conhecimento:\n{contexto}"}
        ]
    )
    return resposta

### Teste

In [65]:
pergunta = "Quero resetar a senha do gerente João, cpf 12345678901 e sou do RH"
contexto = obter_contexto_rag(pergunta)

resposta = chamar_agente(pergunta, contexto)

choice = resposta.choices[0]

if choice.finish_reason == "tool_calls":
    print("Decidiu executar:", choice.message.tool_calls[0].function.name)
    print("Parâmetros:", choice.message.tool_calls[0].function.arguments)
else:
    print("Resposta textual:", choice.message.content)

Este procedimento descreve como um colaborador pode solicitar a redefinição de sua própria senha.

Restrições:
- O colaborador deve ser o titular da conta.
- O acesso à ferramenta de autosserviço de senhas deve estar habilitado.
- Disponível de segunda a sexta.
- Caso não consiga concluir sozinho, o suporte pode auxiliar em horário comercial (08:00–18:00).

Passo a passo:
1. Acesse o portal de autosserviço (senha.empresa.com).
2. Clique em “Esqueci minha senha”.
3. Preencha as informações solicitadas (CPF, e-mail ou matrícula).
4. Escolha o método de verificação: SMS, e-mail alternativo ou token autenticador.
5. Após validação, defina uma nova senha conforme as políticas da empresa.
6. Aguarde até 15 minutos para propagação no ambiente.

Observações:
- Caso o portal não esteja acessível, abra um chamado no Service Desk.
- Em nenhuma hipótese a senha deve ser fornecida por terceiros.
Decidiu executar: resetar_senha
Parâmetros: {"cpf":"12345678901","tipo_usuario":"gerente"}
